In [ ]:
## 설치
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,843 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,994 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/univers

In [ ]:
## 초기화
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [ ]:
import pyspark
from pyspark.sql import DataFrame, SparkSession

In [ ]:
## Session 생성
spark = (
    SparkSession
    .builder
    .appName("First Session")
    .getOrCreate()
)

spark

In [ ]:
spark.sparkContext.getConf().getAll()

[('spark.driver.port', '40901'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.memory', '4g'),
 ('s

In [ ]:
conf = spark.sparkContext._conf.setAll([
     ('spark.executor.memory', '2g'),
     ('spark.app.name', 'Spark Updated Conf'),
     ('spark.executor.cores', '4'),
     ('spark.cores.max', '4'),
     ('spark.driver.memory','2g')
    ])

## Session 생성
spark = (
    SparkSession
    .builder
    .config(conf=conf)
    .getOrCreate()
)

spark

In [ ]:
spark.sparkContext.getConf().getAll()

[('spark.executor.memory', '2g'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.driver.port', '43103'),
 ('spark.executor.id', 'driver'),
 (

In [ ]:
## df 생성
df = spark.range(500).toDF("number")

In [ ]:
df.printSchema()

root
 |-- number: long (nullable = false)



In [ ]:
df.show()

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows



### Transformation

In [ ]:
df.select("number")

DataFrame[number: bigint]

In [ ]:
import pyspark.sql.functions as f

In [ ]:
## 덧셈과 곱셈
df_ = df.select((f.col("number") * 2 + 10).alias("number"))
df_

DataFrame[number: bigint]

In [ ]:
df = df.withColumn('number2', df["number"] * 2 + 10)
df

DataFrame[number: bigint, number2: bigint]

In [ ]:
df.select(df["number"].cast("short").alias('number_short'), "number2")

DataFrame[number_short: smallint, number2: bigint]

In [ ]:
df.filter(df["number"] > 3)

DataFrame[number: bigint, number2: bigint]

### SparkSQL

In [ ]:
## View 생성
df.createOrReplaceTempView("df_number")

In [ ]:
spark.sql("""SELECT * FROM df_number;""").show()

+------+-------+
|number|number2|
+------+-------+
|     0|     10|
|     1|     12|
|     2|     14|
|     3|     16|
|     4|     18|
|     5|     20|
|     6|     22|
|     7|     24|
|     8|     26|
|     9|     28|
|    10|     30|
|    11|     32|
|    12|     34|
|    13|     36|
|    14|     38|
|    15|     40|
|    16|     42|
|    17|     44|
|    18|     46|
|    19|     48|
+------+-------+
only showing top 20 rows



### Action

In [ ]:
df.collect()

[Row(number=0, number2=10),
 Row(number=1, number2=12),
 Row(number=2, number2=14),
 Row(number=3, number2=16),
 Row(number=4, number2=18),
 Row(number=5, number2=20),
 Row(number=6, number2=22),
 Row(number=7, number2=24),
 Row(number=8, number2=26),
 Row(number=9, number2=28),
 Row(number=10, number2=30),
 Row(number=11, number2=32),
 Row(number=12, number2=34),
 Row(number=13, number2=36),
 Row(number=14, number2=38),
 Row(number=15, number2=40),
 Row(number=16, number2=42),
 Row(number=17, number2=44),
 Row(number=18, number2=46),
 Row(number=19, number2=48),
 Row(number=20, number2=50),
 Row(number=21, number2=52),
 Row(number=22, number2=54),
 Row(number=23, number2=56),
 Row(number=24, number2=58),
 Row(number=25, number2=60),
 Row(number=26, number2=62),
 Row(number=27, number2=64),
 Row(number=28, number2=66),
 Row(number=29, number2=68),
 Row(number=30, number2=70),
 Row(number=31, number2=72),
 Row(number=32, number2=74),
 Row(number=33, number2=76),
 Row(number=34, number2=

In [ ]:
df.take(3)

[Row(number=0, number2=10),
 Row(number=1, number2=12),
 Row(number=2, number2=14)]

In [ ]:
df.write.parquet("content/df_dummy.parquet", mode="overwrite")

In [ ]:
df.rdd.getNumPartitions()

2

In [ ]:
import pandas as pd
pd.read_parquet("/content/content/df_dummy.parquet/part-00001-374d0b56-877b-4385-a54b-cbb576d328f5-c000.snappy.parquet")

,number,number2
0,250,510
1,251,512
2,252,514
3,253,516
4,254,518
...,...,...
245,495,1000
246,496,1002
247,497,1004
248,498,1006


In [ ]:
df.toPandas()

,number,number2
0,0,10
1,1,12
2,2,14
3,3,16
4,4,18
...,...,...
495,495,1000
496,496,1002
497,497,1004
498,498,1006


In [ ]:
spark.read.parquet("content/df_dummy.parquet")

DataFrame[number: bigint, number2: bigint]

### UDF

In [ ]:
from pyspark.sql.functions import PandasUDFType
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import IntegerType
import pandas as pd


# @pandas_udf(IntegerType(), PandasUDFType.SCALAR)
@pandas_udf("short")
def double(number:pd.Series) -> pd.Series:

    return number*2

df.select(double("number")).show()

+--------------+
|double(number)|
+--------------+
|             0|
|             2|
|             4|
|             6|
|             8|
|            10|
|            12|
|            14|
|            16|
|            18|
|            20|
|            22|
|            24|
|            26|
|            28|
|            30|
|            32|
|            34|
|            36|
|            38|
+--------------+
only showing top 20 rows

